In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
from pathlib import Path

project_root = Path.cwd()
sys.path.insert(0, str(project_root))

from prepare_params import prepare_production_params

print("Modules imported successfully")
print(f"Current working directory: {os.getcwd()}")


kml = "Sample Points\AfforestationSItesFixed.kml"
shp = "Sample Points\FieldPoints32_2018.shp"
tmx = "Sample Points\TMX\TML_pipeline_100mbuffer.kml"
cbc = "Sample Points\ColdwaterBCregion.kml"

print(kml)
print(shp)
print(tmx)
print(cbc)

Modules imported successfully
Current working directory: C:\Users\aradar\LEAF Files\LEAF_runner
Sample Points\AfforestationSItesFixed.kml
Sample Points\FieldPoints32_2018.shp
Sample Points\TMX\TML_pipeline_100mbuffer.kml
Sample Points\ColdwaterBCregion.kml


In [2]:
# TEMPLATE 1: Single Year Monthly Mosaics
# Use Case: Generate monthly mosaics for specific months in one year
# Example: Summer months (June, July, August) of 2023
# Customize these parameters as needed:

ProdParams = {
    # ============ REQUIRED PARAMETERS ============
    'sensor': 'S2_SR',                    # Sensor type
    'unit': 2,                   # A data unit code (1 or 2 for TOA or surface reflectance)    
    'nbYears': -1,            # positive int for annual product, or negative int for monthly product
    #'year': 2023,                          # CHANGE THIS: Year to process
    #'months': [8,9,10],                   # CHANGE THIS: Months to process (1-12)
    
    
    # ============ REGION PARAMETERS ============
    'regions': cbc,  # CHANGE THIS: Path to your KML/SHP file
    #'file_variables': ['TARGET_FID', 'AsssD_1', 'AsssD_2'], # CHANGE THIS: id, start_date, end_date (None for dates if they don't exist)
    #'file_variables': ['id', 'begin', 'end'],
    'file_variables': ['system:index', None, None],
    'regions_start_index': 0,              # CHANGE THIS: Start at this region index
    'regions_end_index': None,             # CHANGE THIS: End at this index (None = all)
    
    # ============ BUFFER PARAMETERS ============
    #'spatial_buffer_m': 100,             # UNCOMMENT & CHANGE: Buffer in meters around regions
    #'temporal_buffer': [[50, 90], [-10, 10], [100, 130]],          # UNCOMMENT & CHANGE: [days_before, days_after]
    'temporal_buffer': [["2025-08-01", "2025-08-31"]],
    #'num_years': 3,
    
    
    # ============ OUTPUT PARAMETERS ============
    'resolution': 10,                      # Resolution in meters
    'projection': 'EPSG:3979',             # Coordinate projection
    'prod_names': ['mosaic'],    #['LAI', 'FCOVER', 'fAPAR', 'Albedo'], 
    #'bands': ['blue', 'green', 'red', 'rededge1', 'rededge2', 'rededge3', 'nir08', 'swir16', 'swir22'],
    'out_folder': 'E:/S2_mosaics_runner_2026/Coldwater_10m/S2_test_10m',
    'out_datatype': 'int16'
}

CompParams = {
  "number_workers":32,     
  "debug"       : True,
  "entire_tile" : False, 
  "nodes"       : 1,
  "node_memory" : "16G",
  'chunk_size': {'x': 512, 'y': 512}
}


print("\nconfigured")


configured


In [3]:
# Run the parameter preparation
result = prepare_production_params(ProdParams, CompParams)
'''
if result:
    print("\n" + "="*80)
    print(" SUCCESS - Parameters validated and ready!")
    print("="*80)
    print(f"\n Summary:")
    print(f"  Regions: {len(result['ProdParams']['regions'])}")
    print(f"  Time windows: {len(result['ProdParams']['start_dates'])}")
    print(f"  Total tasks: {len(result['ProdParams']['regions']) * len(result['ProdParams']['start_dates'])}")
    print(f"  Output folder: {result['ProdParams']['out_folder']}")
    print(f"\n Processing log saved to: {result['ProdParams']['out_folder']}/polygon_processing_log.csv")
    print("\n" + "="*80)
    print("Next step: Call Production.py with the validated parameters")
    print("="*80)
else:
    print("\n" + "="*80)
    print(" FAILED - Parameter validation failed")
    print("="*80)
    print("Check the error messages above for details.") '''


PARAMETER PREPARATION

Step 1/4: Validating input parameters...
✓ All parameters valid

Step 2/4: Loading regions from files...
<handle_regions_from_file> Detected file-based regions input: Sample Points\ColdwaterBCregion.kml
<handle_regions_from_file> Loading regions...
<handle_regions_from_file> Override mode: Setting all regions to 1 date window(s)
<handle_regions_from_file> Region region0: 1 windows [['2025-08-01', '2025-08-31']]
<handle_regions_from_file> Loaded 1 regions from file

Step 3/4: Skipping temporal window generation (no year/months or start_dates/end_dates found)
           Region-specific dates from file will be used if available.

Step 4/4: Validating polygons...

POLYGON VALIDATION AND FILTERING

Polygon validation log saved to: E:/S2_mosaics_runner_2026/Coldwater_10m/S2_test_10m\polygon_processing_log.csv

Validation Summary:
   Total regions: 1
   Valid for processing: 1
   Skipped (zero area): 0

Proceeding with 1 valid region(s) out of 1 total


PARAMETER PREPA

'\nif result:\n    print("\n" + "="*80)\n    print(" SUCCESS - Parameters validated and ready!")\n    print("="*80)\n    print(f"\n Summary:")\n    print(f"  Regions: {len(result[\'ProdParams\'][\'regions\'])}")\n    print(f"  Time windows: {len(result[\'ProdParams\'][\'start_dates\'])}")\n    print(f"  Total tasks: {len(result[\'ProdParams\'][\'regions\']) * len(result[\'ProdParams\'][\'start_dates\'])}")\n    print(f"  Output folder: {result[\'ProdParams\'][\'out_folder\']}")\n    print(f"\n Processing log saved to: {result[\'ProdParams\'][\'out_folder\']}/polygon_processing_log.csv")\n    print("\n" + "="*80)\n    print("Next step: Call Production.py with the validated parameters")\n    print("="*80)\nelse:\n    print("\n" + "="*80)\n    print(" FAILED - Parameter validation failed")\n    print("="*80)\n    print("Check the error messages above for details.") '

In [4]:
# View processing log (if pandas is available)
try:
    import pandas as pd
    if result and result['processing_log'] is not None:
        print("\n Processing Log Preview:")
        print("=" * 80)
        display(result['processing_log'].head(10))
        print(f"\nTotal entries: {len(result['processing_log'])}")
except ImportError:
    print("pandas not available - install it to view the processing log in this cell")
    print("You can still view the CSV file directly in the output folder")


 Processing Log Preview:


,region_id,area_m2,will_process,status,skip_reason
0,region0,37913071.11,True,VALID,N/A



Total entries: 1


In [5]:
import os
import sys
import geopandas as gpd

#Get the absolute path to the parent of current working directory 
cwd    = os.getcwd()
source_path = os.path.join(cwd, 'source')
sys.path.append(source_path)
sys.path

from Production import main

print(ProdParams)

{'sensor': 'S2_SR', 'unit': 2, 'nbYears': -1, 'regions': {'region0': {'type': 'Polygon', 'coordinates': [[[-120.83748713702258, 50.040446746261715], [-120.73897660873169, 50.040446746261715], [-120.73897660873169, 50.08882798523277], [-120.83748713702258, 50.08882798523277], [-120.83748713702258, 50.040446746261715]]]}}, 'file_variables': ['system:index', None, None], 'regions_start_index': 0, 'regions_end_index': None, 'temporal_buffer': [['2025-08-01', '2025-08-31']], 'resolution': 10, 'projection': 'EPSG:3979', 'prod_names': ['mosaic'], 'out_folder': 'E:/S2_mosaics_runner_2026/Coldwater_10m/S2_test_10m', 'out_datatype': 'int16', 'region_start_dates': {'region0': ['2025-08-01']}, 'region_end_dates': {'region0': ['2025-08-31']}}


In [ ]:
main(ProdParams, CompParams)


<get_time_window> one of required keys is not exist to form time.
<MosaicProduction> User defined input parameters:
sensor: S2_SR
unit: 2
nbYears: -1
regions: {'region0': {'type': 'Polygon', 'coordinates': [[[-120.83748713702258, 50.040446746261715], [-120.73897660873169, 50.040446746261715], [-120.73897660873169, 50.08882798523277], [-120.83748713702258, 50.08882798523277], [-120.83748713702258, 50.040446746261715]]]}}
resolution: 10
projection: EPSG:3979
prod_names: ['mosaic']
out_folder: E:/S2_mosaics_runner_2026/Coldwater_10m/S2_test_10m
out_datatype: int16
region_start_dates: {'region0': ['2025-08-01']}
region_end_dates: {'region0': ['2025-08-31']}
SsrData: {'NAME': 'S2_SR', 'SSR_CODE': 21, 'DATA_UNIT': 2, 'GAIN': 0.0001, 'OFFSET': 0, 'ALL_BANDS': ['blue', 'green', 'red', 'rededge1', 'rededge2', 'rededge3', 'nir08', 'swir16', 'swir22'], '10M_BANDS': ['blue', 'green', 'red', 'nir08', 'swir16', 'swir22'], 'SIX_BANDS': ['blue', 'green', 'red', 'nir08', 'swir16', 'swir22'], 'NoA_BAND

C:\Users\aradar\AppData\Local\anaconda3\envs\leaftoolbox2\Lib\site-packages\distributed\client.py:3164: UserWarning: Sending large graph of size 16.29 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(



<one_mosaic> Final mosaic bands: ['blue', 'green', 'red', 'rededge1', 'rededge2', 'rededge3', 'nir08', 'swir16', 'swir22', 'scl', 'date', 'score']
